Estimating the size of the TESS/whatever dataset

In [ ]:
# All guesses.
years = 8 
sample_interval_min = 5
pixels_per_sample = 10 * 10
bytes_per_pixel = 4
stars = 2000

# Estimating the size.
minutes = years * 365 * 24 * 60
samples_per_star = minutes / sample_interval_min
total_size = samples_per_star * pixels_per_sample * bytes_per_pixel * stars

print(f'{total_size:,} bytes')

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

In [ ]:
DATA = Path.cwd().parent / "data" / "CONFIRMED"

In [ ]:
K2_3 = DATA / "K2-3.csv"

In [ ]:
df = pd.read_csv(K2_3)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
plt.figure(figsize=(12, 4))
plt.title("K2-3")
plt.plot(df["time"], df["flux"], "k-")
plt.show()

# Copying some code from astropy to learn

## Estimating periods

In [ ]:
t = df["time"]
y = df["flux"]
dy = df["flux_err"]

In [ ]:
from utils import auto_max_min_period, autoperiod

In [ ]:
minimum_period, maximum_period, total_duration = auto_max_min_period(t)

In [ ]:
minimum_period, maximum_period, total_duration

In [ ]:
periods = autoperiod(minimum_period, maximum_period, total_duration)
print(len(periods))

In [ ]:
import sys

from tqdm import tqdm

In [ ]:
def new_bls(t, y, dy):

    def normalize(y):
        y -= np.mean(y)
        y /= np.std(y)
        return y

    def autophase(period, duration):
        return np.arange(0, period, period / duration)

    def model(t, y, w, p, d, phi):
        trel = t - np.min(t)
        is_transit = (np.fmod(trel, p) >= phi) & (np.fmod(trel, p) <= phi + d)
        r = np.sum(w * is_transit)
        s = np.sum(w * y * is_transit)
        wx = np.sum(w * y * y)

        # if r * (1 - r) == 0:
        #     return np.inf

        return wx - (s**2) / (r * (1 - r) + sys.float_info.epsilon)

    w = 1.0 / dy**2
    assert np.fabs(np.sum(w)) > sys.float_info.epsilon
    w = w / np.sum(w)

    y = normalize(y)

    best_d = np.inf
    best_period = None
    best_phi = None

    periods = autoperiod(*auto_max_min_period(t))
    periods = np.linspace(10, 30, 500)
    for p in tqdm(periods):
        durations = np.linspace(0.01, 0.05, 50) * p
        for d in durations:
            phase = autophase(period=p, duration=d)
            for phi in phase:
                d_value = model(t, y, w, p, d, phi)
                if d_value < best_d:
                    best_d = d_value
                    best_period = p
                    best_phi = phi

    return best_period, best_phi, best_d

In [ ]:
real_period = 50
real_phase = 5
real_duration = 0.1 * real_period
real_diff = 0.05

threshold = np.cos(np.pi * real_duration / real_period)

t = np.linspace(0, 400, 4000)
y = (np.cos(2.0 * np.pi \
        * (t - real_phase - real_duration / 2.0) / real_period) > threshold) \
    .astype(float)
y = 1.0 - real_diff * y
dy = 0.01 * np.ones(t.shape)

In [ ]:
plt.plot(t, y)

In [ ]:
period, phi, d = new_bls(t, y, dy)

In [ ]:
period, phi, d

In [ ]:
def old_bls(time, flux, flux_err, periods, durations):
    best_period = None
    best_phase = None
    best_D = np.inf
    best_indices = None
    
    # time = np.array(time)
    # flux = np.array(flux)
    # flux_err = np.array(flux_err)
    
    weights = 1 / flux_err**2
    weights /= np.sum(weights)
    x = flux - np.mean(flux)
    x /= np.std(x)
    wx = np.sum(np.multiply(weights, np.multiply(x, x)))

    for period in periods:
        for duration in durations:
            for phase in np.linspace(0, period, int(period / duration)):
                in_transit = (time % period >= phase) & (time % period < phase + duration)
                r = np.sum(weights[in_transit])
                if r == 0 or r == 1:
                    continue
                s = np.sum(weights[in_transit] * x[in_transit])
                
                D = wx - (s**2 / (r * (1 - r)))
                
                if D < best_D:
                    best_D = D
                    best_period = period
                    best_phase = phase
                    best_indices = np.where(in_transit)
    
    return best_period, best_phase, best_indices

In [ ]:
best_period, best_phase, best_indices = old_bls(t, y, dy, periods, durations)

In [ ]:
def bls(time, flux, flux_err, periods, durations, num_bins=200):
    best_period = None
    best_phase = None
    best_D = np.inf
    best_indices = None

    for period in periods:
        for duration in durations:
            for phase in np.linspace(0, period, int(period / duration)):
                r = duration / period
                weights = 1 / flux_err**2
                x = flux - np.mean(flux)
                
                # Bin the folded time series
                folded_time = (time % period) - phase
                folded_time[folded_time < 0] += period  # Ensure all times are positive
                binned_time = np.linspace(0, period, num_bins)
                bin_indices = np.digitize(folded_time, binned_time) - 1
                
                bin_sums = np.zeros(num_bins)
                bin_weights = np.zeros(num_bins)
                bin_fluxes = np.zeros(num_bins)
                
                for i in range(num_bins):
                    bin_mask = (bin_indices == i)
                    bin_sums[i] = np.sum(weights[bin_mask] * x[bin_mask])
                    bin_weights[i] = np.sum(weights[bin_mask])
                    bin_fluxes[i] = np.sum(weights[bin_mask] * x[bin_mask]**2)
                
                # Calculate the BLS statistic
                in_transit_bins = (binned_time >= 0) & (binned_time < duration)
                s = np.sum(bin_sums[in_transit_bins])
                D = np.sum(bin_fluxes[in_transit_bins]) - (s**2 / (r * (1 - r)))
                
                if D < best_D:
                    best_D = D
                    best_period = period
                    best_phase = phase
                    best_indices = np.where(in_transit_bins[bin_indices])
    
    return best_period, best_phase, best_indices

In [ ]:
periods = np.linspace(15,25,100)
duration = np.linspace(0.01, 0.2, 100)

In [ ]:
best_period, best_phase, best_indices = bls(df.time, df.flux, df.flux_err, periods, duration)

In [ ]:
print(f"Best period: {best_period}")
print(f"Best phase: {best_phase}")
print(f"Indices: {best_indices}")